In [0]:
from google.colab import drive
drive.mount('/content/drive')

## IMPORTING LIBRARIES

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

## DATALOADER

### LOAD/DOWNLOAD DATASET

In [0]:
! cp -r '/content/drive/My Drive/Assignment 5 Dataset'

# !gdown https://drive.google.com/uc?id=1j9bZ2drKlr_btZEBR-melf2RHUWhAPSv
# !unzip /content/Assignment5Data.zip 

data_dir = '/content/Assignment 5 Dataset'


### Calculating Mean and Std for Normalizing Images

In [0]:
# train_data = datasets.ImageFolder(data_dir + '/train', transform=transforms.ToTensor())

# mean = torch.zeros(size=(1,3))
# meansq = torch.zeros(size=(1,3))
# # meansq = 
# for data,_ in train_data:
#   mean += data.mean([1,2])
#   meansq += data.pow(2).mean([1,2])

# mean = mean/len(train_data)
# meansq = meansq/len(train_data)
# std = torch.sqrt(meansq-mean.pow(2))
# print(mean)
# print(std)

# tensor([[0.4905, 0.4905, 0.4906]])
# tensor([[0.2494, 0.2494, 0.2494]])


### DATALOADER

In [0]:
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.4905, 0.4905, 0.4906],
                                                            [0.2494, 0.2494, 0.2494])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4905, 0.4905, 0.4906],
                                                           [0.2494, 0.2494, 0.2494])])


#pass transform here-in
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
val_data = datasets.ImageFolder(data_dir + '/validation', transform=test_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

#data loaders
batch_size =32
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)



print("Classes: ")
class_names = train_data.classes
print(class_names)

In [0]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(trainloader))
show_databatch(inputs, classes)

## LOAD MODEL

In [0]:
def initialize_model(model,freeze):
    if model == 'res18':
        res18 = models.resnet18(pretrained=True)
        fc1_in = res18.fc.in_features

        if freeze == 'all':
            for param in res18.parameters():
                param.requires_grad=False
        elif freeze == 'partial':
            for param in list(res18.parameters())[:-17]:
                param.requires_grad=False
        elif freeze == 'none':
            pass


    elif model == 'vgg16':
        vgg16 == models.vgg16(pretrained=True)
        fc1_in = vgg16.classifier[0].in_features

        if freeze == 'all':
            for param in vgg16.features.parameters():
                param.requires_grad = False
        elif freeze == 'partial':
            for param in list(vgg16.features.parameters())[:-6]:
                param.requires_grad = False
        elif freeze == 'none':
            pass


    fc1_out = 5*10+100
    fc2_in = fc1_out
    fc2_out = 2
    features = [nn.Linear(fc1_in,fc1_out,bias=True),nn.ReLU(inplace=True),nn.Dropout(p=0.5,inplace=False),nn.Linear(fc2_in,fc2_out,bias=True)]

    if model=='res18':
        res18.fc = nn.Sequential(*features)
        return res18
    if model == 'vgg16':
        vgg16.classifier = nn.Sequential(*features)
        if pretarined_weights != None:
          vgg16.load_state_dict(torch.load(pretrained_weights)['state_dict'])
        return vgg16

net = 'vgg16'
freeze = 'none'
model = initialize_model(net,freeze)



### Loading from Pretrained Weights

In [0]:
path = '/content/trained_models/vgg16_ft_89.73_0.18_11.pth'
model.load_state_dict(torch.load(path)['state_dict'])

### Loss fucntion and optimizer

In [0]:
Epochs = 50
criterion = nn.CrossEntropyLoss()
lr = 1e-5
optimizer = optim.SGD(vgg16.parameters(), lr=lr, momentum=0.9)

## TRAINING

In [0]:
os.mkdir('/content/trained_models')
path = '/content/trained_models'

In [0]:
from tqdm import tqdm
# from pytorchtools import EarlyStopping
import torch.nn.functional as F

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
val_loss_min= 1e6
val_acc_prev = 0.0

epoch_lr = []
epoch_tacc = []
epoch_tloss = []
epoch_vacc= []
epoch_vloss = []

# epoch_lr =[1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5,1e-5]

for epoch in range(Epochs):  # loop over the dataset multiple times
    val_accuracy =  0.0
    train_accuracy = 0.0

    epoch_lr.append(lr)
    running_loss = 0.0
    # pbar = tqdm(total = len(trainloader))
    
    # pbar = tqdm(len(trainloader))
    
    
    model.train()
    pbar = tqdm(enumerate(trainloader),position=0,leave=True)
    # for i, data in pbar:
    for i,data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = model(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        # print(i," ",loss.item())
        pred = torch.argmax(F.softmax(outputs),dim=1)
        # pred = torch.argmax(outputs,dim=1).float()
        # print("PREDICTION",pred)
        # print("LABELS",labels)
        correct = (pred==labels).float().sum()
        # print("CORRECT",correct)
        accuracy = correct*100/inputs.shape[0]
        train_accuracy += correct


        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.1f}%'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data.item(),accuracy),refresh=False)
        
    print("\nTraining Loss of Epoch ",epoch," is :",running_loss)
    tacc= train_accuracy*100/len(trainloader.dataset)
    print("Training Accuracy of Epoch ",epoch," is :",tacc.item(),"\n\n")
    

    epoch_tacc.append(tacc.item())
    epoch_tloss.append(running_loss)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valloader),position=0,leave=True)
    # for i, data in pbar:
    for i,data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        # optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = model(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        # loss.backward()                     #----> backward pass
        # optimizer.step()                    #----> weights update

        # print statistics
        val_loss += loss.item()
        # print(i," ",loss.item())
        
        pred = torch.argmax(outputs,dim=1)
        correct = (pred==labels).float().sum()
        # print(correct)
        accuracy = correct*100/inputs.shape[0]
        val_accuracy += correct
        
        pbar.set_description(
            'Validation Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.1f}%'.format(
                epoch, i * len(inputs), len(valloader.dataset),
                100. * i / len(valloader),
                loss.data.item(),accuracy),refresh=False)
        
    print("\nValidation Loss of Epoch ",epoch," is :",val_loss)
    acc = val_accuracy*100/len(valloader.dataset)
    print("Validation Accuracy of Epoch ",epoch," is :",acc.item(),"\n\n")

    epoch_vloss.append(val_loss)
    epoch_vacc.append(acc.item())

    if val_loss < val_loss_min:
        checkpoint = {
                'epoch': epoch + 1,
                'valid_loss_min': val_loss,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }

        val_loss_min = val_loss
        # save_ckp(checkpoint, False, checkpoint_path, best_model_path)


            # pbar.update()
            # pbar()
        torch.save(checkpoint, os.path.join(path,'vgg16_ft_{:.2f}_{:.2f}_{}.pth'.format(acc,loss,epoch)))

    delta = abs(acc.item() - val_acc_prev)
    val_acc_prev = acc.item()

    if (delta < 0.4)and(lr<0.1):
        lr = lr*10
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
print('Finished Training')

## EVALUATION

In [0]:
save_path= '/content/drive/My Drive/vgg_results'

### Plot Loss and Accuracy Curves

In [0]:
import os.path as osp

# from matplotlib.ticker import MaxNLocator
# Epochs=21
## LOSS AND ACCURACY CURVES ##
plt.figure()
plt.plot(range(Epochs),np.array(epoch_tloss).reshape(Epochs),color='k',label='Train')
plt.plot(range(Epochs),np.array(epoch_vloss).reshape(Epochs),color='b',label='Validation')

# ax=plt.figure().gca()
# ax.xaxis.set_major_locator(MaxNLocator(integer=True))

plt.title('Loss Curves')
plt.xlabel('epochs')
plt.ylabel('Loss')
plt.ylim(0,100)
plt.xlim(1,50)
# plt.xticks(range(1,22))
plt.legend()

plt.savefig(osp.join(save_path,'Loss Curves.png'),bbox_inches='tight')
################ ACCURACY##################

plt.figure()
plt.plot(range(Epochs),epoch_tacc,color='k',label='Train')
plt.plot(range(Epochs),epoch_vacc,color='b',label='Validation')
plt.ylim(0,100)
plt.xlim(1,50)
# plt.xticks(range(1,22))

plt.title('Accuracy Curves')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig(osp.join(save_path,'Accuracy Curves.png'),bbox_inches='tight')

### Evaluate

In [0]:
## CONFUSION MATRIX ##


correct = 0
total = 0
predicted = []
gt = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        pred = torch.argmax(outputs,dim=1)
        correct += (pred==labels).float().sum()
        
        predicted.extend(torch.Tensor.cpu(pred).numpy())
        gt.extend(torch.Tensor.cpu(labels).numpy())
        # _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # correct += (pred == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


from sklearn.metrics import confusion_matrix
import pandas as pd
import itertools

# y_true = [2, 0, 2, 2, 0, 1]
# y_pred = [0, 0, 2, 2, 0, 2]
# out
# pred=np.argmax(out,axis=1)
cfm = confusion_matrix(gt, predicted)
recall = cfm[1,1]/cfm[1,:].sum()
precision = cfm[1,1]/cfm[:,1].sum()
f1 = (2*precision*recall)/(precision+recall)
print("F1 SCORE: ",f1)
# cfm = pd.DataFrame(cfm,index=np.arange(10),columns=np.arange(10))
# sns.heatmap(cfm,annot=True)

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plot_confusion_matrix(cfm, classes=np.arange(2))
plt.savefig(osp.join(save_path,'cfm_train.png'),bbox_inches='tight')



        
